<a href="https://colab.research.google.com/github/rositata/scraping-book/blob/main/praktik_scraping_books.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
# Import Library
!pip install beautifulsoup4 requests
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin
import time
import random
from tqdm import tqdm

**Keterangan:**

requests (mengambil halaman website)

BeautifulSoup (memproses HTML)

pandas (membuat tabel DataFrame)

time dan random (memberi jeda acak agar scraping tidak dianggap spam)

In [24]:
base = "https://books.toscrape.com/"
catalog = base + "catalogue/"
books = []

def get_soup(url):
    res = requests.get(url)
    if res.status_code == 200:
        return BeautifulSoup(res.text, "html.parser")
    else:
        return None

Pada tahap ini Kita mulai scraping dari halaman pertama, nanti otomatis lanjut ke halaman berikutnya sampai (1000 buku total).

In [25]:
# Ambil semua link kategori
soup = get_soup(base)
cats = [base + c['href'] for c in soup.select(".side_categories a")[1:]]

Menentukan kolom data yang akan diambil (title, price, availability, rating, url).

In [30]:
# Loop setiap kategori
for cat in cats:
    next_page = cat
    while next_page:
        soup = get_soup(next_page)
        if not soup:
            break

        for item in soup.select("article.product_pod"):
            title = item.h3.a["title"]
            price = item.select_one(".price_color").text.replace("£", "").replace("Â", "")
            stock = item.select_one(".availability").text.strip()
            rating = item.select_one(".star-rating")["class"][1]
            link = catalog + item.h3.a["href"].replace("../../../", "")
            books.append({
                "title": title,
                "price": float(price),
                "availability": stock,
                "rating": rating,
                "url": link
            })

        # cek tombol next
        nxt = soup.select_one("li.next a")
        if nxt:
            next_page = next_page.rsplit('/', 1)[0] + '/' + nxt['href']
        else:
            next_page = None

        time.sleep(random.uniform(0.1, 0.25))

In [31]:
df = pd.DataFrame(books)
print("Hasil Scraping dengan 1000 Jumlah buku:", len(df))
df.head()

Hasil Scraping dengan 1000 Jumlah buku: 1000


,title,price,availability,rating,url
0,It's Only the Himalayas,45.17,In stock,Two,https://books.toscrape.com/catalogue/its-only-...
1,Full Moon over Noahâs Ark: An Odyssey to Mou...,49.43,In stock,Four,https://books.toscrape.com/catalogue/full-moon...
2,See America: A Celebration of Our National Par...,48.87,In stock,Three,https://books.toscrape.com/catalogue/see-ameri...
3,Vagabonding: An Uncommon Guide to the Art of L...,36.94,In stock,Two,https://books.toscrape.com/catalogue/vagabondi...
4,Under the Tuscan Sun,37.33,In stock,Three,https://books.toscrape.com/catalogue/under-the...


Setelah melakukan loopung otomatis kategori secara berurutan tiap halamanya. Data tiap buku dimasukan di list books dengan tujuan untuk data buku (judul, harga, stok, rating, link) yang berhasil di-scrape bisa disimpan sementara.

In [32]:
df.to_csv("Hasil Scraping dengan 1000 Jumlah buku.csv", index=False)
print("Data berhasil disimpan sebagai books_toscrape.csv")

Data berhasil disimpan sebagai books_toscrape.csv


Menyimpan hasil scraping ke file dengan csv untuk mempermudah ke GitHub dan dibuka di Excel.